In [ ]:
import nltk
import re
from nltk.stem import PorterStemmer
import os
import json
from charset_normalizer import from_path
from gensim.corpora import Dictionary
import numpy as np
import pickle
import random
import pickle
import gc
import sys
from tqdm.auto import tqdm 

STEMMER = PorterStemmer()
ACTUAL_PATH = os.getcwd()
PATH_20N = os.path.join(ACTUAL_PATH, "data/20news-18828")
PATH_BAC = os.path.join(ACTUAL_PATH, "data/BAC/blogs")
PATH_FINAL_FILES = os.path.join(ACTUAL_PATH, "data/final_files")
GRUPO = "03"

## I. Read the files and build two large consolidate files that are the union of all the documents in 20N and BAC.

### UPLOAD_FILES

#### UPLOADING 20N 

In [2]:
NEW_20N_FILE = os.path.join(PATH_FINAL_FILES, "20N.jsonl")

mayor_folders_20N = os.listdir(PATH_20N)
dictionary = {}

with open(NEW_20N_FILE, "w", encoding="utf-8", errors="replace") as f_n:
    for folder in mayor_folders_20N:
        minor_files_path = os.path.join(PATH_20N, folder)
        minor_files = os.listdir(minor_files_path)
        for file in minor_files:
            file_path = os.path.join(minor_files_path, file)
            with open(file_path, "r", encoding="utf-8", errors="replace") as f:
                text = f.read().strip()

            record = {"id": file, "theme": folder, "source": "20N", "text": text}
            unit = folder + file
            if file in dictionary.keys():
                dictionary[unit] += 1
            else:
                dictionary[unit] = 1
            f_n.write(json.dumps(record, ensure_ascii=False) + "\n")

In [3]:
## TODO: Verificar si los encodings estan bien

#### UPLOADING BAC

In [4]:
NEW_BAC_FILE = os.path.join(PATH_FINAL_FILES, "BAC.jsonl")
mayor_folders_BAC = os.listdir(PATH_BAC)
with open(NEW_BAC_FILE, "w", encoding="utf-8", errors="replace") as f_n:
    for file in mayor_folders_BAC:
        post_num = 0
        file_path = os.path.join(PATH_BAC, file)
        with open(file_path, "r", encoding="utf-8", errors="replace") as f:
            text = f.read().strip()
            text = re.sub(r"</?Blog>", "", text)
        post_list = text.split("<post>")
        for post in post_list:
            post = post.strip().replace("</post>", "")
            record = {"id": file, "post_num": post_num, "source": "BAC", "text": post}
            f_n.write(json.dumps(record, ensure_ascii=False) + "\n")
            post_num += 1

## II. Tokenize by sentence

In [5]:
import nltk
import re

In [6]:
# def get_sentences(text:str, k: int) -> list[list]:
#     sentences_list = []
#     text = text.split()
#     last = 0
#     for i in range(k,len(text)+1):
#         sentences = text[last:i]
#         last +=1
#         text_slice = " ".join(sentences)
#         text_slice_token = preprocess_text(text_slice)
#         sentences_list.append(text_slice_token)
#     return sentences_list
def preprocess_text(text: str) -> list[str]:

    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9\s]", " ", text, flags=re.I | re.A | re.MULTILINE)
    text = re.sub(r"\d+", "NUM", text)
    text = "<s> " + text
    text = text + " </s>"
    text = text.strip().split()
    # tokens = [STEMMER.stem(word) for word in text]
    return text


def divide_sentences(text: str) -> list[str]:
    sentences = nltk.sent_tokenize(text)
    return sentences

### Examples

In [7]:
text = "This is sentence one. Here is another! And number 123."
sentences = divide_sentences(text)
sentences

['This is sentence one.', 'Here is another!', 'And number 123.']

In [8]:
processed = [preprocess_text(s) for s in sentences]

print(processed)

[['<s>', 'this', 'is', 'sentence', 'one', '</s>'], ['<s>', 'here', 'is', 'another', '</s>'], ['<s>', 'and', 'number', 'NUM', '</s>']]


### Tokenizing_full_text

In [9]:
import random
import pickle
import gc

def save_pickle(data, filename):
    filepath = os.path.join(PATH_FINAL_FILES, filename)
    with open(filepath, "wb") as f:
        pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

word_count = {}
with open(os.path.join(PATH_FINAL_FILES, "20N.jsonl"), "r") as f:
    for line in f:
        line = json.loads(line)
        for word in preprocess_text(line["text"]):
            word_count[word] = word_count.get(word, 0) + 1
word_count = {word: count for word, count in word_count.items() if count == 1}

sentencias = []
with open(os.path.join(PATH_FINAL_FILES, "20N.jsonl"), "r") as f:
    for line in f:
        line = json.loads(line)
        sentences = divide_sentences(line["text"])
        for sentence in sentences:
            pre_process = preprocess_text(sentence)
            tokens = [word if word not in word_count else "<UNK>" for word in pre_process]
            sentencias.append(tokens)
            # print(sentence, pre_process)
random.seed(42)
random.shuffle(sentencias)
index_to_split = int(0.8 * len(sentencias))
train_sentences = sentencias[:index_to_split]
test_sentences = sentencias[index_to_split:]
save_pickle(train_sentences, f"20N_{GRUPO}_training.pkl")
save_pickle(test_sentences, f"20N_{GRUPO}_testing.pkl")

del sentencias
del train_sentences
del test_sentences
gc.collect()

0

In [10]:



def save_pickle(data, filename):
    filepath = os.path.join(PATH_FINAL_FILES, filename)
    with open(filepath, "wb") as f:
        pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

word_count = {}
with open(os.path.join(PATH_FINAL_FILES, "BAC.jsonl"), "r") as f:
    for line in f:
        line = json.loads(line)
        for word in preprocess_text(line["text"]):
            word_count[word] = word_count.get(word, 0) + 1
word_count = {word: count for word, count in word_count.items() if count == 1}

sentencias = []
with open(os.path.join(PATH_FINAL_FILES, "BAC.jsonl"), "r") as f:
    for line in f:
        line = json.loads(line)
        sentences = divide_sentences(line["text"])
        for sentence in sentences:
            pre_process = preprocess_text(sentence)
            tokens = [word if word not in word_count else "<UNK>" for word in pre_process]
            sentencias.append(tokens)
            # print(sentence, pre_process)
random.seed(42)
random.shuffle(sentencias)
split_idx = int(0.8 * len(sentencias))
train_sentences = sentencias[:split_idx]
test_sentences = sentencias[split_idx:]
save_pickle(train_sentences, f"BAC_{GRUPO}_training.pkl")
save_pickle(test_sentences, f"BAC_{GRUPO}_testing.pkl")

del sentencias
del train_sentences
del test_sentences
gc.collect()

0

In [11]:
# import os
# import json
# import random
# import pickle

In [12]:
# def save_pickle(data, filename):
#     filepath = os.path.join(PATH_FINAL_FILES, filename)
#     with open(filepath, "wb") as f:
#         pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

# def process_and_split(jsonl_file, grupo, train_ratio=0.8, seed=42):
#     random.seed(seed)
#     train_sentences = []
#     test_sentences = []

#     with open(jsonl_file, "r", encoding="utf-8") as f:
#         for line in f:
#             line = json.loads(line)
#             for sentence in divide_sentences(line["text"]):
#                 pre_process = preprocess_text(sentence)
#                 # Decidir en el momento si va a train o test
#                 if random.random() < train_ratio:
#                     train_sentences.append(pre_process)
#                 else:
#                     test_sentences.append(pre_process)

#     save_pickle(train_sentences, f"BAC_{grupo}_training.pkl")
#     save_pickle(test_sentences, f"BAC_{grupo}_testing.pkl")

# # Uso
# process_and_split(os.path.join(PATH_FINAL_FILES, "BAC.jsonl"), GRUPO)

## IV. Calcular N Gramas

### Calcular Unigramas

#### 20N

In [25]:
from tqdm import tqdm
_HAS_TQDM = True

In [ ]:
class UnigramModel:
    def __init__(self, filename, file_is_training = True):
        print(filename)
        file = self.get_pickle(filename)
        if file_is_training:
            self.word_counter_20N = {}
            for sentence in file:
                for word in sentence:
                    self.word_counter_20N[word] = self.word_counter_20N.get(word, 0) + 1
        else:
            self.word_counter_20N = file
        self.total_words = sum(self.word_counter_20N.values())
        self.V = len(self.word_counter_20N)
    
    def get_pickle(self, filename: str) -> list[str]:
        filepath = os.path.join(PATH_FINAL_FILES, filename)
        with open(filepath, "rb") as f:
            sentences = pickle.load(f)
        return sentences
    def generate_unigrams(self, filename):
        filepath = os.path.join(PATH_FINAL_FILES, filename)
        probabilities_unigram = {
            self.get_prob(word)
            for word in self.word_counter_20N.keys()
        }
        with open(filepath, "wb") as f:
            pickle.dump(probabilities_unigram, f, protocol=pickle.HIGHEST_PROTOCOL)
    def get_prob(self, word: str):
        if word.lower() in self.word_counter_20N.keys():
            prob = self.word_counter_20N[word] / self.total_words
        else:
            prob = self.word_counter_20N["<UNK>"] / self.total_words
        return prob
    
class BigramModel:
    def __init__(self, filename, file_is_training=True, alpha=1.0):
        self.alpha = alpha
        data = self._load_pickle(filename)

        if file_is_training:
            self.dictionary = Dictionary(data)
            specials = ["<UNK>"]
            self.dictionary.add_documents([[tok] for tok in specials])

            self.V = len(self.dictionary)

            # Solo diccionarios
            self.matrix = {}              # (w1_idx, w2_idx) -> int
            self.unigram_counts = {}      # w_idx -> int
            self.row_sums = {}            # w1_idx -> total saliente (int)

            for sentence in data:
                # (opcional) añade <BOS>/<EOS> aquí si lo deseas
                for i in range(len(sentence)):
                    w_idx = self._word_index(sentence[i])
                    self.unigram_counts[w_idx] = self.unigram_counts.get(w_idx, 0) + 1
                    if i < len(sentence) - 1:
                        w_next_idx = self._word_index(sentence[i + 1])
                        key = (w_idx, w_next_idx)
                        self.matrix[key] = self.matrix.get(key, 0) + 1
                        self.row_sums[w_idx] = self.row_sums.get(w_idx, 0) + 1

        else:
            self.dictionary = data["dictionary"]
            self.V = data["V"]
            self.matrix = dict(data["matrix"])
            self.unigram_counts = dict(data["unigram_counts"])
            self.row_sums = dict(data["row_sums"])
            self.alpha = data.get("alpha", 1.0)

        self.N_words = sum(self.unigram_counts.values())
        self.N = sum(self.matrix.values())

    def _word_index(self, word: str) -> int:
        tid = self.dictionary.token2id.get(word)
        if tid is None:
            tid = self.dictionary.token2id["<UNK>"]
        return tid

    def _load_pickle(self, filename: str):
        filepath = os.path.join(PATH_FINAL_FILES, filename)
        with open(filepath, "rb") as f:
            return pickle.load(f)

    def save_model(self, filename: str):
        payload = {
            "dictionary": self.dictionary,
            "V": self.V,
            "matrix": dict(self.matrix),
            "unigram_counts": dict(self.unigram_counts),
            "row_sums": dict(self.row_sums),
            "alpha": self.alpha,
        }
        filepath = os.path.join(PATH_FINAL_FILES, filename)
        with open(filepath, "wb") as f:
            pickle.dump(payload, f, protocol=pickle.HIGHEST_PROTOCOL)

    def token_of(self, idx: int) -> str:
        return self.dictionary.id2token.get(idx, "<UNK>")

    def get_prob(self, words: list[str]) -> float:

        m_i = self._word_index(words[0])
        m_j = self._word_index(words[1])

        alpha = self.alpha

        c_w1 = self.unigram_counts.get(m_i, 0)
        c_bigram = self.matrix.get((m_i, m_j), 0)
        row_sum = self.row_sums.get(m_i, 0)

        p_w1 = (c_w1 + alpha) / (self.N_words + alpha * self.V)
        p_cond = (c_bigram + alpha) / (row_sum + alpha * self.V)

        return np.log(p_w1) + np.log(p_cond)

    def generate_bigrams(self, filename: str, show_progress: bool = True):
        keys = list(self.matrix.keys())
        print("len pasado")
        probabilities_bigram = {}
        print("iniciaciando")

        iterator = tqdm(keys, desc="Generando bigramas", unit="bigrama",
                                total=len(keys), leave=True, file=sys.stdout)

        for i, j in iterator:
            w1 = self.token_of(i)
            w2 = self.token_of(j)
            probabilities_bigram[(w1, w2)] = self.get_prob([w1, w2])

        filepath = os.path.join(PATH_FINAL_FILES, filename)
        with open(filepath, "wb") as f:
            pickle.dump(probabilities_bigram, f, protocol=pickle.HIGHEST_PROTOCOL)
    
class TrigramModel:
    """
    Modelo de trigramas siguiendo el estilo de BigramModel.
    - Usa Dictionary de gensim con token <UNK>.
    - Estructuras de conteo vía diccionarios:
        * unigram_counts[i]                = c(w1)
        * matrix_bigram[(i,j)]             = c(w1,w2)
        * row_sums[i]                      = sum_j c(i,j)   (denominador de P(w2|w1))
        * matrix_trigram[(i,j,k)]          = c(w1,w2,w3)
        * pair_sums[(i,j)]                 = sum_k c(i,j,k) (denominador de P(w3|w1,w2))
    - Suavizado aditivo alpha.
    - Probabilidad conjunta: log P(w1) + log P(w2|w1) + log P(w3|w1,w2).
    - Guardado/carga con pickle.
    """

    def __init__(self, filename, file_is_training=True, alpha=1.0):
        self.alpha = float(alpha)

        data = self._load_pickle(filename)

        if file_is_training:
            # Construye diccionario y añade <UNK> como en BigramModel
            self.dictionary = Dictionary(data)
            self.dictionary.add_documents([["<UNK>"]])

            self.V = len(self.dictionary)

            # Diccionarios de conteo
            self.unigram_counts = {}       # i -> c(i)
            self.matrix_bigram  = {}       # (i,j) -> c(i,j)
            self.row_sums       = {}       # i -> sum_j c(i,j)

            self.matrix_trigram = {}       # (i,j,k) -> c(i,j,k)
            self.pair_sums      = {}       # (i,j) -> sum_k c(i,j,k)

            # Una sola pasada por oración
            for sentence in data:
                ids = [self._word_index(w) for w in sentence]
                L = len(ids)

                for t in range(L):
                    i = ids[t]
                    # unigrama
                    self.unigram_counts[i] = self.unigram_counts.get(i, 0) + 1

                    # bigrama
                    if t + 1 < L:
                        j = ids[t + 1]
                        key2 = (i, j)
                        self.matrix_bigram[key2] = self.matrix_bigram.get(key2, 0) + 1
                        self.row_sums[i] = self.row_sums.get(i, 0) + 1

                        # trigrama
                        if t + 2 < L:
                            k = ids[t + 2]
                            key3 = (i, j, k)
                            self.matrix_trigram[key3] = self.matrix_trigram.get(key3, 0) + 1
                            self.pair_sums[(i, j)] = self.pair_sums.get((i, j), 0) + 1

        else:
            # Carga desde modelo ya entrenado
            self.dictionary     = data["dictionary"]
            self.V              = data["V"]
            self.unigram_counts = dict(data["unigram_counts"])
            self.matrix_bigram  = dict(data["matrix_bigram"])
            self.row_sums       = dict(data["row_sums"])
            self.matrix_trigram = dict(data["matrix_trigram"])
            self.pair_sums      = dict(data["pair_sums"])
            self.alpha          = data.get("alpha", 1.0)

        # Totales
        self.N_words    = sum(self.unigram_counts.values())
        self.N_bigrams  = sum(self.matrix_bigram.values())
        self.N_trigrams = sum(self.matrix_trigram.values())

    # ---------- utilidades de IO / diccionario ----------

    def _load_pickle(self, filename: str):
        filepath = os.path.join(PATH_FINAL_FILES, filename)
        with open(filepath, "rb") as f:
            return pickle.load(f)

    def save_model(self, filename: str):
        payload = {
            "dictionary": self.dictionary,
            "V": self.V,
            "unigram_counts": dict(self.unigram_counts),
            "matrix_bigram":  dict(self.matrix_bigram),
            "row_sums":       dict(self.row_sums),
            "matrix_trigram": dict(self.matrix_trigram),
            "pair_sums":      dict(self.pair_sums),
            "alpha": self.alpha,
        }
        filepath = os.path.join(PATH_FINAL_FILES, filename)
        with open(filepath, "wb") as f:
            pickle.dump(payload, f, protocol=pickle.HIGHEST_PROTOCOL)

    def token_of(self, idx: int) -> str:
        return self.dictionary.id2token.get(idx, "<UNK>")

    def _word_index(self, word: str) -> int:
        tid = self.dictionary.token2id.get(word)
        if tid is None:
            tid = self.dictionary.token2id["<UNK>"]
        return tid

    # ---------- probabilidad ----------

    def get_prob(self, words: list[str]) -> float:

        i = self._word_index(words[0])  # w1
        j = self._word_index(words[1])  # w2
        k = self._word_index(words[2])  # w3

        V = self.V
        a = self.alpha

        c_w1       = self.unigram_counts.get(i, 0)
        c_w1w2     = self.matrix_bigram.get((i, j), 0)
        c_w1w2w3   = self.matrix_trigram.get((i, j, k), 0)

        denom_w1       = self.N_words
        denom_w2gw1    = self.row_sums.get(i, 0)
        denom_w3gw1w2  = self.pair_sums.get((i, j), 0)

        p_w1            = (c_w1     + a) / (denom_w1      + a * V)
        p_w2_given_w1   = (c_w1w2   + a) / (denom_w2gw1   + a * V)
        p_w3_given_w1w2 = (c_w1w2w3 + a) / (denom_w3gw1w2 + a * V)

        return np.log(p_w1) + np.log(p_w2_given_w1) + np.log(p_w3_given_w1w2)


    def generate_trigrams(self, filename: str, show_progress: bool = True):
        """
        Guarda en pickle un dict:
        { (w1, w2, w3) (str): log_prob_total }
        solo para trigramas OBSERVADOS (claves en matrix_trigram).
        """
        keys = list(self.matrix_trigram.keys())
        iterator = keys
        if show_progress and _HAS_TQDM:
            iterator = tqdm(keys, desc="Generando trigramas", unit="trigrama",
                            total=len(keys), leave=True, file=sys.stdout)

        trigram_logprobs = {}
        for (i, j, k) in iterator:
            w1, w2, w3 = self.token_of(i), self.token_of(j), self.token_of(k)
            trigram_logprobs[(w1, w2, w3)] = self.get_prob([w1, w2, w3])

        filepath = os.path.join(PATH_FINAL_FILES, filename)
        with open(filepath, "wb") as f:
            pickle.dump(trigram_logprobs, f, protocol=pickle.HIGHEST_PROTOCOL)

    # ---------- (opcional) distribución condicional para muestreo ----------
    def next_word_logprobs(self, w1: str, w2: str) -> dict[str, float]:
        """
        Devuelve { w3: log P(w3|w1,w2) } sobre TODO el vocabulario (con suavizado).
        Usa pair_sums como denominador (óptimo).
        """
        i = self._word_index(w1)
        j = self._word_index(w2)

        a = self.alpha
        V = self.V
        denom = self.pair_sums.get((i, j), 0)

        out = {}
        for k in range(V):
            c = self.matrix_trigram.get((i, j, k), 0)
            p = (c + a) / (denom + a * V)
            out[self.token_of(k)] = float(np.log(p))
        return out

In [32]:
def get_pickle(filename: str) -> list[str]:
    filepath = os.path.join(PATH_FINAL_FILES, filename)
    with open(filepath, "rb") as f:
        sentences = pickle.load(f)
    return sentences

In [ ]:
import math
def perplexity_unigram(model, filename):
    sentences = model.get_pickle(filename)
    log_sum = 0.0
    T = 0
    for s in sentences:
        for w in s:
            p = model.get_prob(w)
            if p == 0.0:
                return float("inf")
            log_sum += math.log(p)
            T += 1
    return math.exp(-log_sum / T) if T else float("inf")

def perplexity_bigram_from_model(model, filename):
    sentences = model._load_pickle(filename)
    log_sum = 0.0
    T = 0
    for s in sentences:
        for prev, cur in zip(s[:-1], s[1:]):
            log_p = model.get_prob([prev, cur])  
            log_sum += log_p
            T += 1
    return math.exp(-log_sum / T) if T else float("inf")

def perplexity_trigram_from_model(model, filename):
    sentences = model._load_pickle(filename)
    log_sum = 0.0
    T = 0
    for s in sentences:
        for prev_2, prev_1 ,cur in zip(s[:-1], s[1:], s[2:]):
            log_p = model.get_prob([prev_2, prev_1, cur])  
            log_sum += log_p
            T += 1
    return math.exp(-log_sum / T) if T else float("inf")

In [ ]:
modelo_de_unigramas = UnigramModel(f"20N_{GRUPO}_training.pkl")
#modelo_de_unigramas.generate_unigrams(f"20N_{GRUPO}_unigrams.pkl")
print("Perplejidad de 20N training Unigram",perplexity_unigram(modelo_de_unigramas,f"20N_{GRUPO}_testing.pkl"))

modelo_de_unigramas = UnigramModel(f"BAC_{GRUPO}_training.pkl")
#modelo_de_unigramas.generate_unigrams(f"BAC_{GRUPO}_unigrams.pkl")
print("Perplejidad de BAC training Unigram",perplexity_unigram(modelo_de_unigramas,f"BAC_{GRUPO}_testing.pkl"))


20N_03_training.pkl


1029.422742712471

In [ ]:
modelo_de_unigramas = BigramModel(f"20N_{GRUPO}_training.pkl")
#modelo_de_unigramas.generate_bigrams(f"20N_{GRUPO}_bigrams.pkl")
print("Perplejidad de 20N training Bigram",perplexity_bigram_from_model(modelo_de_unigramas,f"20N_{GRUPO}_testing.pkl"))


BAC_03_training.pkl


735.7160239678001

In [ ]:
modelo_de_unigramas = BigramModel(f"BAC_{GRUPO}_training.pkl")
#modelo_de_unigramas.generate_bigrams(f"BAC_{GRUPO}_bigrams.pkl")
print("terminado biramas bac",perplexity_unigram(modelo_de_unigramas,f"BAC_{GRUPO}_testing.pkl"))


In [ ]:
modelo_de_unigramas = TrigramModel(f"20N_{GRUPO}_training.pkl")
#modelo_de_unigramas.generate_trigrams(f"20N_{GRUPO}_trigrams.pkl")
print("terminado trigramas 20 ",perplexity_trigram_from_model(modelo_de_unigramas,f"20N_{GRUPO}_testing.pkl"))

3418203.2430977863

In [ ]:
modelo_de_unigramas = TrigramModel(f"BAC_{GRUPO}_training.pkl")
#modelo_de_unigramas.generate_trigrams(f"BAC_{GRUPO}_trigrams.pkl")
print("terminado trigramas 20 ",perplexity_trigram_from_model(modelo_de_unigramas,f"BAC{GRUPO}_testing.pkl"))

## Trash

In [28]:
modelo_de_unigramas = TrigramModel(f"20N_{GRUPO}_training.pkl")
modelo_de_unigramas.generate_trigrams(f"20N_{GRUPO}_trigrams.pkl")
print("terminado trigramas 20 ")

Generando trigramas: 100%|██████████| 2275391/2275391 [00:11<00:00, 203161.69trigrama/s]
terminado trigramas 20 


In [ ]:
modelo_de_unigramas = TrigramModel(f"BAC_{GRUPO}_training.pkl")
modelo_de_unigramas.generate_trigrams(f"BAC_{GRUPO}_trigrams.pkl")

Generando trigramas: 100%|██████████| 38147426/38147426 [03:15<00:00, 195581.59trigrama/s]
terminado trigramas bac


In [ ]:
modelo_de_bigrama = BigramModel(f"20N_{GRUPO}_training.pkl")

In [ ]:
modelo_de_trigrama = TrigramModel(f"20N_{GRUPO}_training.pkl")

In [ ]:
modelo_de_bigrama.get_prob(["rationalist",  "press",  "association"  ])

-35.54000515220461

In [ ]:
len(modelo_de_unigramas.matrix)

1095576

In [ ]:
len(modelo_de_unigramas.matrix)

1095576

In [ ]:
modelo_de_unigramas.predict_prob('when')

0.001482720662526471

In [ ]:
train_20N

[['<s>',
  'so',
  'when',
  'you',
  've',
  'got',
  'multi',
  'tasking',
  'you',
  'want',
  'to',
  'increase',
  'performance',
  'by',
  'increasing',
  'the',
  'amount',
  'of',
  'overlapping',
  'you',
  'do',
  '</s>'],
 ['<s>',
  'my',
  'sister',
  'lyuda',
  'was',
  'in',
  'yerevan',
  'at',
  'the',
  'time',
  '</s>'],
 ['<s>',
  'in',
  'early',
  'march',
  'the',
  'idf',
  'allowed',
  'representatives',
  'of',
  'b',
  'tselem',
  'the',
  'israeli',
  'information',
  'center',
  'for',
  'human',
  'rights',
  'in',
  'the',
  'territories',
  'to',
  'visit',
  'tulkarm',
  'prison',
  'but',
  'denied',
  'them',
  'access',
  'to',
  'the',
  'interrogation',
  'wing',
  '</s>'],
 ['<s>',
  'oh',
  'yes',
  'the',
  'organization',
  's',
  'national',
  'ecclesisatical',
  'advisor',
  'is',
  'catholic',
  'politician',
  'cardinal',
  'john',
  'j',
  'o',
  'connor',
  'of',
  'new',
  'york',
  '</s>'],
 ['<s>',
  'you',
  'really',
  'need',
  'to',

In [ ]:
"20N_{GRUPO}_unigrams"

#### BAC

### Calcular Bigramas

#### 20N

#### BAC

### Calcular Trigamas

#### 20N

#### BAC